In [ ]:
%load_ext autoreload
%autoreload 2

# MTBO Reizman Study

In [2]:
from summit.strategies import MTBO, Transform, LHS, Chimera
from summit.benchmarks import ReizmanSuzukiEmulator, BaumgartnerCrossCouplingEmulator
from summit.utils.dataset import DataSet
from summit.domain import *
import summit
import pathlib
import pandas as pd

## Create DataSet

In [3]:
# Path to data
summit_path = pathlib.Path(summit.__file__).parents[0]
data_path = summit_path / "benchmarks" / "experiment_emulator" / "data"

In [4]:
# Import Reizman-Suzuki Data
datasets = [DataSet.read_csv(data_path / f"reizman_suzuki_case{i}_train_test.csv") for i in range(1,5)]
for i, dataset in enumerate(datasets):
    dataset[('task', 'METADATA')] = i
datasets[0].tail(5)

,catalyst,t_res,temperature,catalyst_loading,ton,yield,task
91,P1-L6,189.7,110,1.127,6.6,7.4,0
92,P1-L4,600,110,1.052,65.4,68.7,0
93,P1-L3,600,65.3,1.106,5.1,5.6,0
94,P2-L1,600,30,0.496,1.1,0.6,0
95,P1-L2,189.7,110,2.513,21.1,52.9,0


In [5]:
exp = ReizmanSuzukiEmulator(case=1)
exp.domain

Name,Type,Description,Values
catalyst,"categorical, input",Catalyst type - different ligands,8 levels
t_res,"continuous, input",Residence time in seconds (s),"[60,600]"
temperature,"continuous, input",Reactor temperature in degrees Celsius (ºC),"[30,110]"
catalyst_loading,"continuous, input",Catalyst loading in mol%,"[0.5,2.5]"
ton,"continuous, minimize objective",Turnover number - moles product generated divided by moles catalyst used,"[0,200]"
yield,"continuous, maximize objective",Yield,"[0,100]"


In [6]:
lhs = LHS(exp.domain)
conditions = lhs.suggest_experiments(5)
exp.run_experiments(conditions)

AttributeError: 'CategoricalVariable' object has no attribute 'enc'

In [ ]:
exp.reset()
hierarchy = {"yield": {"hierarchy": 0, "tolerance": 0.5}, "ton": {"hierarchy": 1, "tolerance": 1}}
transform = Chimera(exp.domain, hierarchy=hierarchy)
strategy = MTBO(exp.domain, 
                pretraining_data=datasets[1],
                transform=transform,
                categorical_method="one-hot", 
                task=0)
r = summit.Runner(strategy=strategy, experiment=exp, max_iterations=10)
r.run()

In [ ]:
exp.pareto_plot(colorbar=True)

In [ ]:
exp.data

In [3]:
new_exp = ReizmanSuzukiEmulator(case=2)
# new_exp.train(dataset=datasets[1], verbose=False)

In [4]:
lhs = LHS(new_exp.domain)
conditions = lhs.suggest_experiments(10)
new_exp.run_experiments(conditions)

DomainError: Variable Variable(name=catalyst, description=Catalyst type - different ligands) is not one of the possible variable types (continuous or categorical).

In [6]:
pd_cn = BaumgartnerCrossCouplingEmulator()
pd_cn.domain

Name,Type,Description,Values
catalyst,"categorical, input",Catalyst type,3 levels
base,"categorical, input",Base,4 levels
base_equivalents,"continuous, input",Base equivalents,"[1.0,2.5]"
temperature,"continuous, input",Temperature in degrees Celsius (ºC),"[30,100]"
t_res,"continuous, input",residence time in seconds (s),"[60,1800]"
yld,"continuous, maximize objective",Yield,"[0.0,1.0]"


In [8]:
lhs = LHS(pd_cn.domain, categorical_method="descriptors")
conditions = lhs.suggest_experiments(10)
pd_cn.run_experiments(conditions)

,catalyst,base,base_equivalents,temperature,t_res,yld,computation_t,experiment_t,strategy,M2,M2_cat,area,area_cat
0,tBuXPhos,TMG,1.675,68.5,1713.0,0.558063,966.787776,0.038967,LHS,198.078405,586.466845,28.59991,95.049495
1,tBuXPhos,TMG,1.375,96.5,1365.0,0.811928,966.787776,0.029236,LHS,185.067785,514.631105,78.70129,70.299455
2,tBuXPhos,TMG,2.275,54.5,843.0,0.108371,966.787776,0.028499,LHS,172.057165,478.713235,39.73355,76.486965
3,tBuXPhos,TMG,2.425,75.5,1017.0,0.524719,966.787776,0.028885,LHS,204.583715,658.302585,50.86719,119.799535
4,tBuXPhos,TMG,2.125,61.5,1191.0,0.315026,966.787776,0.029264,LHS,217.594335,550.548975,67.56765,113.612025
5,tBuXPhos,TMG,1.075,89.5,1539.0,0.734635,966.787776,0.028474,LHS,165.551855,694.220455,34.16673,82.674475
6,tBuXPhos,TMG,1.825,82.5,321.0,0.341075,966.787776,0.028474,LHS,178.562475,622.384715,56.43401,125.987045
7,tBuXPhos,TMG,1.975,33.5,669.0,0.037157,966.787776,0.028860,LHS,211.089025,766.056195,45.30037,88.861985
8,tBuXPhos,TMG,1.525,40.5,147.0,0.036645,966.787776,0.028159,LHS,191.573095,801.974065,62.00083,107.424515
9,tBuXPhos,TMG,1.225,47.5,495.0,0.046041,966.787776,0.026539,LHS,224.099645,730.138325,73.13447,101.237005
